https://stackoverflow.com/questions/77145106/how-to-get-historical-price-of-options-with-strike-price-y-relative-to-being-set

#### This works only on options that have yet to expire. It doesnt work on options that have already expired.

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import datetime as dt
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import warnings 

In [2]:
# Settings the warnings to be ignored 
warnings.filterwarnings('ignore') 

%matplotlib ipympl

In [3]:
today_date = dt.datetime.today().strftime('%Y-%m-%d')

### This function is for getting options data across all strikes and expiry dates

In [4]:
def fetch_all_option_data(ticker): 
    """
    Fetch option chain data for a given stock ticker. 
    """
    Ticker = yf.Ticker(ticker)
    expirations = Ticker.options
    
    option_data = pd.DataFrame() 
    
    for expiration in expirations:
        
        option_chain = Ticker.option_chain(expiration) 
        
        calls = option_chain.calls
        calls['Option Type'] = "Call" 
        
        puts = option_chain.puts
        puts['Option Type'] = "Put" 
        
        options = pd.concat([calls, puts])
        options['Expiration Date'] = pd.to_datetime(expiration) + pd.DateOffset(hours=23, minutes=59, seconds=59)
        
        option_data = pd.concat([option_data, options])
    option_data["Days to Expiration"] = (option_data['Expiration Date'] - dt.datetime.today()).dt.days + 1 
    
    return option_data

### This function is for getting option data for a specific expiry date and strike

In [5]:
def get_option_data(stock_symbol, expiration_date, option_type, strike):
    stock = yf.Ticker(stock_symbol)
    option_chain = stock.option_chain(expiration_date)
    options = getattr(option_chain, "calls" if option_type.startswith("call") else "puts")
    option_data = options[options["strike"] == strike]
    return option_data


### This function returns the history for a particular option contract x days before expiration

In [6]:
def get_option_history_data(contract_symbol, days_before_expiration=30):     #Can change the days before expiration here
    option = yf.Ticker(contract_symbol)
    option_info = option.info
    option_expiration_date = dt.datetime.fromtimestamp(option_info["expireDate"])

    start_date = option_expiration_date - dt.timedelta(days=days_before_expiration)
    option_history = option.history(start=start_date)
    return option_history

### This function is if you want specific information for a specfic contract on a specific date

In [7]:
def main(stock_symbol = "AAPL",expiration_date = "2024-09-06", option_type = "call", strike = 220.0):
    
    option_data = get_option_data(stock_symbol, expiration_date, option_type, strike)
    for i, od in option_data.iterrows():
        contract_symbol = od["contractSymbol"]
        option_history = get_option_history_data(contract_symbol)
        first_option_history = option_history.iloc[0]
        first_option_history_date = option_history.index[0]
        first_option_history_close = first_option_history["Close"]
        print("For {}, the closing price was ${:.2f} on {}.".format(
            contract_symbol,
            first_option_history_close,
            first_option_history_date
        ))


In [8]:
# Remember to change the expiration date and strike

if __name__ == "__main__":
    main()

For AAPL240906C00220000, the closing price was $3.20 on 2024-08-07 00:00:00-04:00.


In [9]:
# Remember to change the expiration date and strike

Info=main(stock_symbol = "TSLA",expiration_date = "2024-09-06", option_type = "call", strike = 170.0)

For TSLA240906C00170000, the closing price was $28.60 on 2024-08-07 00:00:00-04:00.


In [10]:
# This cell gets all the available expirations for a ticker

stock = yf.Ticker("TSLA")
expirations = stock.options
expirations

('2024-08-23',
 '2024-08-30',
 '2024-09-06',
 '2024-09-13',
 '2024-09-20',
 '2024-09-27',
 '2024-10-18',
 '2024-11-15',
 '2024-12-20',
 '2025-01-17',
 '2025-02-21',
 '2025-03-21',
 '2025-06-20',
 '2025-08-15',
 '2025-09-19',
 '2025-12-19',
 '2026-01-16',
 '2026-06-18',
 '2026-12-18')

In [11]:
# This cell gets all the options expiring on a certain date for that ticker

option_chain = stock.option_chain(expirations[1])    #Second most immediate date
option_chain

Options(calls=         contractSymbol             lastTradeDate  strike  lastPrice     bid  \
0   TSLA240830C00075000 2024-07-12 14:44:51+00:00    75.0     176.10  124.55   
1   TSLA240830C00085000 2024-08-02 18:32:54+00:00    85.0     124.09  130.15   
2   TSLA240830C00095000 2024-08-02 18:34:55+00:00    95.0     113.95  120.15   
3   TSLA240830C00100000 2024-08-16 19:55:06+00:00   100.0     116.81  115.15   
4   TSLA240830C00105000 2024-08-16 19:44:16+00:00   105.0     111.93  110.30   
..                  ...                       ...     ...        ...     ...   
73  TSLA240830C00410000 2024-08-15 19:24:07+00:00   410.0       0.01    0.00   
74  TSLA240830C00420000 2024-08-16 13:54:18+00:00   420.0       0.01    0.00   
75  TSLA240830C00430000 2024-08-08 18:12:29+00:00   430.0       0.02    0.00   
76  TSLA240830C00440000 2024-08-16 13:55:43+00:00   440.0       0.01    0.00   
77  TSLA240830C00450000 2024-08-15 14:59:34+00:00   450.0       0.01    0.00   

       ask     change  pe

In [18]:
# Test cell for getting the history for a particular option contract

option = yf.Ticker('TSLA240830C00440000')
option_info = option.info
option_expiration_date = dt.datetime.fromtimestamp(option_info["expireDate"])
start_date = option_expiration_date - dt.timedelta(days=300)
option_history = option.history(start=start_date,end=today_date,period='1d')
option_history

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2024-07-15 00:00:00-04:00,0.90,0.90,0.83,0.84,5,0.0,0.0
2024-07-17 00:00:00-04:00,0.64,0.64,0.42,0.42,0,0.0,0.0
2024-07-18 00:00:00-04:00,0.41,0.41,0.41,0.41,2,0.0,0.0
2024-07-19 00:00:00-04:00,0.20,0.23,0.20,0.23,2,0.0,0.0
2024-07-23 00:00:00-04:00,0.32,0.32,0.32,0.32,0,0.0,0.0
2024-07-24 00:00:00-04:00,0.04,0.05,0.02,0.02,20,0.0,0.0
2024-07-25 00:00:00-04:00,0.03,0.05,0.03,0.03,15,0.0,0.0
2024-07-30 00:00:00-04:00,0.05,0.05,0.05,0.05,0,0.0,0.0
2024-07-31 00:00:00-04:00,0.03,0.03,0.01,0.01,0,0.0,0.0


In [13]:
# Fetch option data for stock
options_data = fetch_all_option_data("TSLA")

In [14]:
# Gets all the active contracts for that stock
unique_contracts=options_data['contractSymbol'].unique()

In [15]:
len(unique_contracts)

2832

In [16]:
unique_contracts

array(['TSLA240823C00075000', 'TSLA240823C00085000',
       'TSLA240823C00090000', ..., 'TSLA261218P00520000',
       'TSLA261218P00530000', 'TSLA261218P00540000'], dtype=object)

In [ ]:
#This took 8 minutes to run with TSLA

options_data_history_df={}

for option in unique_contracts:
    value=get_option_history_data(contract_symbol=option, days_before_expiration=365)
    options_data_history_df[option]=value

In [ ]:
options_data_history_df